In [1]:
from flask import Flask, render_template, send_file, Response, request, flash, stream_with_context
import base64, zmq
import time
import cv2
import numpy as np
import urllib
import time


from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.embed import components
from bokeh.models import ColumnDataSource

In [2]:
from itertools import cycle
from random import shuffle
import random



def mini_shuffle(array):
    for row in array:
        shuffle(row)
    return array

def make_dummy(number):
    count_dummy = np.linspace(1,4*number,4*number).reshape((number,4))
    mini_shuffle(count_dummy)
    time_dummy = count_dummy+0.5-np.random.random(size=(number,4))
    count_dummy = count_dummy.flatten()
    time_dummy = time_dummy.flatten()
    for i in range(number*4):
        yield count_dummy[i], time_dummy[i]

def stream_sim(number):
    gen1 = make_dummy(number)
    gen2 = make_dummy(number)
    meta_data = {}
    for i in range(number*8):
        if random.randint(0,1):
            meta_data["camera"] = "camera_1"
            meta_data["url"] = "url_1"
            meta_data["counter"], meta_data["timestamp"] = next(gen1)
            meta_data["boxes"] = random.randint(0,10)
            message = "msg from camera 1"
        else:
            meta_data["camera"] = "camera_2"
            meta_data["url"] = "url_2"
            meta_data["counter"], meta_data["timestamp"] = next(gen2)
            message = "msg from camera 2"
            meta_data["boxes"] = random.randint(0,10)

        yield meta_data, message

In [196]:
dummy = stream_sim(25)

In [197]:
dory =Dory()

In [183]:
IPcameras = {}
IPcameras["Gallup@NM"] = "http://207.192.232.2:8000/mjpg/video.mjpg?timestamp=1548280479340"
IPcameras["Agua Fria@NM"] = "http://166.241.180.137/mjpg/video.mjpg?timestamp=1548281679736"
IPcameras["Bolton@CT"] = "http://166.248.11.128/mjpg/video.mjpg?timestamp=1548281838022"
IPcameras["El Paso@TX"] = "http://209.194.208.53/mjpg/video.mjpg?timestamp=1548281942187"
IPcameras["Jamestown@ND"]= "http://64.77.205.67/mjpg/video.mjpg"
IPcameras["Newark@NY"] = "http://166.140.227.198/mjpg/video.mjpg?timestamp=1548369639459"
IPcameras["Brighton@MA"] = "http://128.197.128.161/mjpg/video.mjpg"
IPcameras["KL@Malaysia"] = "http://58.26.96.56/mjpg/video.mjpg"
IPcameras["Taegu@Korea"] = "http://220.73.58.90/mjpg/video.mjpg"
IPcameras["Newark2@NY"] = "http://108.53.114.166/mjpg/video.mjpg"
IPcameras["Louisville@KY"] = "http://74.142.179.46:81/mjpg/video.mjpg?timestamp=1548535207899"
IPcameras["Richmond@VA"] = "http://50.73.9.194/mjpg/video.mjpg"
IPcameras["Rancho@CA"] ="http://166.165.58.225/mjpg/video.mjpg?timestamp=1548608344038"
IPcameras["Fayetteville@NC"] = "http://65.40.167.160/mjpg/video.mjpg"
IPcameras["Newyork"] = "http://67.77.134.186:8001/mjpg/video.mjpg"
IPcameras["SanJose@CA"] = "http://50.252.187.219/mjpg/video.mjpg?timestamp=1548704853592"


In [4]:
from itertools import cycle
import time

def get_img_base64_str(path):
    with open(path, "rb") as imgfp:
        b64str = base64.b64encode(imgfp.read())
    return b64str.decode()

img_str0 = get_img_base64_str("temp/cat-1.jpeg")
img_strs = cycle([
        get_img_base64_str("temp/cat-1.jpeg"),
        get_img_base64_str("temp/cat-2.jpeg"),
        get_img_base64_str("temp/cat-3.jpeg")
    ])

def gentest():
    return 0
    
def cat_render():
    while True:
        time.sleep(.5)
        yield render_template("showimage2.html", img_data=next(img_strs))

In [4]:
app = Flask(__name__)

def stream_template(template_name, **context):
    app.update_template_context(context)
    t = app.jinja_env.get_template(template_name)
    rv = t.stream(context)
    rv.enable_buffering(2)
    time.sleep(1)
    return rv

@app.route('/')
def render_large_template():
    return Response(stream_template('showimage2.html', img_data=img_strs))

In [4]:
app = Flask(__name__)

@app.route('/')
def index():
    return Response(stream_with_context(cat_render()))
#    return render_template('showimage.html')

In [ ]:
app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Feb/2019 17:31:27] "GET / HTTP/1.1" 200 -


In [2]:
def bubble_insert(array, number):
# Insert a new number into the numpy array.
    i = 0
    pop_num = array[-1]
    while number<array[i] and i<len(array):
        i+=1
    array[i+1:] = array[i:-1]
    array[i]=number

    return pop_num

class Dory:
# A very lightweight prototype database in memory. Named after Dolly the goldfish with very short memory.
    def __init__(self, maxsize = 10):
        self.limit = maxsize
        self.record = dict()
# A buffer that saves the data. We want it to maintain a order sorted by the "counter" key 
# but don't want to sort the dictionary every time. The maximum number of the dictionary is "maxsize".
        self.bookkeeper =dict()
        self.output = [None]*3

# A dictionary /that keeps track of the order of the buffer dictionary items.
    
    def look(self, meta_data, message):
        try:
            camera_name = meta_data["camera"]
            timestamp = float(meta_data["timestamp"])
            counter = int(meta_data["counter"])
            boxes = meta_data["boxes"]
            if camera_name not in self.record:
                print("create new")
                self.record[camera_name] ={counter:[timestamp, boxes, message]}
                self.bookkeeper[camera_name] = np.zeros(self.limit).astype("int")
                self.bookkeeper[camera_name][0]=counter
                pop_num=0
            else:
                print("not new")
                self.record[camera_name][counter] = [timestamp, boxes, message]
                pop_num = bubble_insert(self.bookkeeper[camera_name],counter)
                print("pop_num is", pop_num)
                if pop_num>0:
                    self.output = self.record[camera_name][pop_num]
                    self.record[camera_name].pop(pop_num)
        except:
            print("Error occurred during operation")
        return pop_num
    
        def count(self):
# reserved for future improvements.
            pass
            


In [3]:
dummy = open("/Users/Xiaoqing/Insight/CV2capture/Received/SanJose_CA_1.jpg","rb").read()

#IP = "tcp://18.232.52.148:5558"
#IP="tcp://35.175.180.11:5558"
IP = "tcp://34.239.152.41:5558"
#        print('consumer started...Ingest images from ', IP)
context = zmq.Context()
    # recieve work
consumer_receiver = context.socket(zmq.PULL)
consumer_receiver.connect(IP)
     
            
def dory_work(camera):
# Check if the incoming frame come from the camera the user requested. If not, don't show it.
    dory1 = Dory()
    img_bytes = dummy

    while True:
        meta_data = consumer_receiver.recv_json()
        message = consumer_receiver.recv()
        pop_num = dory1.look(meta_data, message)
        if meta_data["camera"]==camera:
            timestamp, boxes, message = dory1.output
            if message is not None:
                buffer = base64.b64decode(message)
            # html can directly read the buffer as image.
                img = cv2.imdecode(np.frombuffer(buffer, np.uint8),cv2.IMREAD_ANYCOLOR)
                overlay_boxes(boxes,img)
            # return to buffer if not modified.
                img_bytes = cv2.imencode('.jpg',img)[1].tobytes()
   #     boxes = meta_data['boxes']
    #    img = cv2.imdecode(np.frombuffer(buffer, np.uint8),cv2.IMREAD_ANYCOLOR)
    #    overlay_boxes(boxes,img)
    #    img_bytes = cv2.imencode('.jpg',img)[1].tobytes()
        yield bytes2display(img_bytes)
        
def work2(camera, dory = Dory(), img_bytes = dummy):
# Check if the incoming frame come from the camera the user requested. If not, don't show it.
 #   dory1 = Dory()
  #  img_bytes = dummy

    meta_data = consumer_receiver.recv_json()
    message = consumer_receiver.recv()
    pop_num = dory1.look(meta_data, message)
    if meta_data["camera"]==camera:
        timestamp, boxes, message = dolly.output
        buffer = base64.b64decode(message)
        img = cv2.imdecode(np.frombuffer(buffer, np.uint8),cv2.IMREAD_ANYCOLOR)
        overlay_boxes(boxes,img)
        img_bytes = cv2.imencode('.jpg',img)[1].tobytes()
   #     boxes = meta_data['boxes']
    #    img = cv2.imdecode(np.frombuffer(buffer, np.uint8),cv2.IMREAD_ANYCOLOR)
    #    overlay_boxes(boxes,img)
    #    img_bytes = cv2.imencode('.jpg',img)[1].tobytes()
    return bytes2display(img_bytes), float(timestamp), len(boxes)
        

def bytes2display(buffer):
    return (b'--frame\r\n'+b'Content-Type: image/jpeg\r\n\r\n' + buffer + b'\r\n') 
    
def overlay_boxes(boxes,image):
# Overlay the detected box on the image. It directly modifies the raw images
# so it should only be used for showing the graph.
    for box in boxes:
        x1, y1, x2, y2 = box[:4]
        x1, y1, x2, y2 = int(x1),int(y1),int(x2),int(y2)
        image[x1:x1+4,y1:y2+1,:] =0
        image[x2:x2+4,y1:y2+1,:] =0
        image[x1:x2+1,y1:y1+4,:] =0
        image[x1:x2+1,y2:y2+4,:] = 0
        image[x1:x1+4,y1:y2+1,0] = 255
        image[x2:x2+4,y1:y2+1,0] = 255
        image[x1:x2+1,y1:y1+4,0] = 255
        image[x1:x2+1,y2:y2+4,0] = 255


In [13]:
%%time
img_bytes=urllib.parse.quote(dummy)

CPU times: user 18.7 ms, sys: 3.19 ms, total: 21.9 ms
Wall time: 22.2 ms


In [5]:
def make_plot():
    x=[88,99,100]
    y=[88,99,100]
    source = ColumnDataSource(dict(x=x, y=y))
    plot = figure(plot_height=200,plot_width = 400)
    plot.line('x', 'y', source=source)
    temp=100
    while True:
        temp+= 1
        x.append(temp)
        y.append(temp*2)
        source.data=dict(x=x, y=y)
#        return components(plot)
        bokeh_plot =components(plot)
        yield bokeh_plot


In [5]:
def my_render():
    yield render_template('showimage2.html', img_data = urllib.parse.quote(base64.b64encode(dummy)))
    while True:
        meta_data = consumer_receiver.recv_json()
        message = consumer_receiver.recv()
        camera_name = meta_data["camera"]
        timestamp = float(meta_data["time"])
        counter = int(meta_data["counter"])
        boxes = meta_data["boxes"]
        yield render_template('showimage2.html', img_data = urllib.parse.quote(message))

def work():
     while True:
        meta_data = consumer_receiver.recv_json()
        message = consumer_receiver.recv()
        camera_name = meta_data["camera"]
        timestamp = float(meta_data["time"])
        counter = int(meta_data["counter"])
        boxes = meta_data["boxes"]
        
        buffer = base64.b64decode(message)
        img = cv2.imdecode(np.frombuffer(buffer, np.uint8),cv2.IMREAD_ANYCOLOR)
        overlay_boxes(boxes,img)
        img_bytes = cv2.imencode('.jpg',img)[1].tobytes()
        
        yield bytes2display(img_bytes)

In [4]:
app = Flask(__name__)
#app.jinja_env.filters['b64d'] = lambda u: base64.b64encode(u).decode()

@app.route('/')
def index():
#    return Response(my_render())
    return render_template('showimage.html'
#    return render_template('showimage2.html', img_data = urllib.parse.quote(buffer))


    
@app.route('/video_feed')
def video_feed():
    camera = 'KL_Malaysia'
    return Response(work(),mimetype='multipart/x-mixed-replace; boundary=frame')
 #   return Response(bytes2display(dummy),mimetype='multipart/x-mixed-replace; boundary=frame')
  #  return render_template("showimage.html",user_image=gen2(test))


#@app.route('/bokeh_feed1')
#def bokeh_feed():
#    return Response(stream_with_context(make_plot()))
#def display_image():
#    full_filename = "static/SanJose_CA_1.jpg"
#    return render_template("index.html",user_image=full_filename)



SyntaxError: invalid syntax (<ipython-input-4-b2a3c48ee1b4>, line 13)

In [ ]:
app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [6]:
Response(make_plot(),mimetype='text/csv')

<Response streamed [200 OK]>

## Webserver

In [ ]:
from itertools import cycle
import asyncio
import base64
import datetime
import json
import random
import websockets

start = time.time()
dummy = open("/Users/Xiaoqing/Insight/CV2capture/Received/SanJose_CA_1.jpg","rb").read()

#IP = "tcp://18.232.52.148:5558"
#IP="tcp://35.175.180.11:5558"
IP = "tcp://34.239.152.41:5558"
context = zmq.Context()
consumer_receiver = context.socket(zmq.PULL)
consumer_receiver.connect(IP)
     

async def trigger(websocket,path):
    while True:
        meta_data = consumer_receiver.recv_json()
        img_msg = consumer_receiver.recv()
        camera_name = meta_data["camera"]
        timestamp = float(meta_data["time"])-start
        counter = int(meta_data["counter"])
        boxes = meta_data["boxes"]
                
        buffer = base64.b64decode(img_msg)
        img = cv2.imdecode(np.frombuffer(buffer, np.uint8),cv2.IMREAD_ANYCOLOR)
        overlay_boxes(boxes,img)
        img_bytes = base64.b64encode(cv2.imencode('.jpg',img)[1].tobytes())
        img_bytes=urllib.parse.quote(img_bytes)
        message = {          
            "img_b64": img_bytes,
            "counter": counter,
            "timestamp":timestamp,
            "boxnum": len(boxes)
        }
        await websocket.send(json.dumps(message))
        await asyncio.sleep(0.02)

start_server = websockets.serve(trigger, '127.0.0.1',5678)

asyncio.get_event_loop().run_until_complete(start_server)
asyncio.get_event_loop().run_forever()

Error in connection handler
Traceback (most recent call last):
  File "/Users/Xiaoqing/anaconda/lib/python3.6/site-packages/websockets/server.py", line 169, in handler
    yield from self.ws_handler(self, path)
  File "<ipython-input-4-563264b0008f>", line 40, in trigger
    await websocket.send(json.dumps(message))
  File "/Users/Xiaoqing/anaconda/lib/python3.6/site-packages/websockets/protocol.py", line 462, in send
    yield from self.ensure_open()
  File "/Users/Xiaoqing/anaconda/lib/python3.6/site-packages/websockets/protocol.py", line 646, in ensure_open
    ) from self.transfer_data_exc
websockets.exceptions.ConnectionClosed: WebSocket connection is closed: code = 1001 (going away), no reason


In [2]:
import os
def consumer():
    IP = "tcp://18.207.129.170:5560"
    print('consumer started...Ingest images from ', IP)
    context = zmq.Context()
    # recieve work
    consumer_receiver = context.socket(zmq.PULL)
    consumer_receiver.connect(IP)
    receiving_dir = './received/'
  #  if not os.path.exists(receiving_dir):
   #     os.makedirs(receiving_dir)
    while True:
        meta_data = consumer_receiver.recv_json()
        saving_path = receiving_dir+meta_data['camera']+'_'+meta_data['counter']+'.jpg'
        message = consumer_receiver.recv()
        with open(saving_path, 'wb') as f:
            buff = base64.b64decode(message)
            f.write(buff)
        print(saving_path, "received and saved")


In [ ]:
consumer()

consumer started...Ingest images from  tcp://18.207.129.170:5560
./received/Richmond_VA_1.jpg received and saved
./received/Richmond_VA_2.jpg received and saved
./received/Richmond_VA_3.jpg received and saved
./received/Newark_NY_1.jpg received and saved
./received/Richmond_VA_4.jpg received and saved
./received/Newark_NY_2.jpg received and saved
./received/Newark_NY_3.jpg received and saved
./received/Richmond_VA_5.jpg received and saved
./received/Newark_NY_4.jpg received and saved
./received/Newark_NY_5.jpg received and saved
./received/Richmond_VA_6.jpg received and saved
./received/Newark_NY_6.jpg received and saved
./received/Newark_NY_7.jpg received and saved
./received/Newark_NY_8.jpg received and saved
./received/Richmond_VA_7.jpg received and saved
./received/Newark_NY_9.jpg received and saved
./received/Newark_NY_10.jpg received and saved
./received/Newark_NY_11.jpg received and saved
./received/Richmond_VA_8.jpg received and saved
./received/Newark_NY_12.jpg received and sa

./received/Richmond_VA_72.jpg received and saved
./received/Richmond_VA_73.jpg received and saved
./received/Richmond_VA_74.jpg received and saved
./received/Richmond_VA_75.jpg received and saved
./received/Richmond_VA_76.jpg received and saved
./received/Richmond_VA_77.jpg received and saved
./received/Richmond_VA_78.jpg received and saved
./received/Richmond_VA_79.jpg received and saved
./received/Richmond_VA_80.jpg received and saved
./received/Richmond_VA_81.jpg received and saved
./received/Richmond_VA_82.jpg received and saved
./received/Richmond_VA_83.jpg received and saved
./received/Richmond_VA_84.jpg received and saved
./received/Richmond_VA_85.jpg received and saved
./received/Richmond_VA_86.jpg received and saved
./received/Richmond_VA_87.jpg received and saved
./received/Richmond_VA_88.jpg received and saved
./received/Richmond_VA_89.jpg received and saved
./received/Richmond_VA_90.jpg received and saved
./received/Richmond_VA_91.jpg received and saved
./received/Richmond_

In [6]:
consumer_receiver.unbind(IP)

ZMQError: No such file or directory

In [3]:
os.getcwd()

'/Users/Xiaoqing/Insight/Raven/flask/flask_app'

In [5]:
import time
time.strftime("%H:%M:%S",time.localtime(time.time()))

'17:22:44'

In [6]:
time.localtime(time.time())

time.struct_time(tm_year=2019, tm_mon=2, tm_mday=10, tm_hour=17, tm_min=22, tm_sec=50, tm_wday=6, tm_yday=41, tm_isdst=0)

In [14]:
24*365

8760

In [25]:
str(int(time.time()*1000))

'1550099287464'

In [26]:
int('1550099287464')

1550099287464